<a href="https://colab.research.google.com/github/HaojiaK/blog/blob/ML_security_and_privacy/Tensorflow_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-privacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from absl import logging 
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow import estimator as tf_estimator
from tensorflow.compat.v1 import estimator as tf_compat_v1_estimator


tf.flags.DEFINE_float('learning_rate',.15,'Learning rate for training')
tf.flags.DEFINE_integer('batch_size',256,'Batch size')
tf.flags.DEFINE_integer('epoch',10,'Number of epochs')

FLAGS = tf.flags.FLAGS

def cnn_model_fn(features, labels, mode):
  """Model function for a CNN"""

  #Define CNN architecture using tf.keras.layers.
  input_layer = tf.reshape(features['x'],[-1,28,28,1])
  y = tf.keras.layers.Conv2D(
      16, 8, strides=2, padding='same', activation='relu').apply(input_layer)
  y = tf.keras.layers.MaxPool2D(2,1).apply(y)
  y = tf.keras.layers.Conv2D(
      32, 4, strides=2, padding='valid', activation='relu').apply(y)
  y = tf.keras.layers.MaxPool2D(2,1).apply(y)
  y = tf.keras.layers.Flatten().apply(y)
  y = tf.keras.layers.Dense(32, activation='relu').apply(y)
  logits = tf.keras.layers.Dense(10).apply(y)
  predicted_labels = tf.argmax(input=logits, axis=1)


  # Calculate loss as a vector and as its average across minibatch
  vector_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits)
  scalar_loss = tf.reduce_mean(vector_loss)

  # Configure the training op (for TRAIN mode)
  if mode == tf_estimator.ModeKeys.TRAIN:
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(FLAGS.learning_rate)
    opt_loss = scalar_loss
    global_step = tf.compat.v1.train.get_global_step()
    train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, train_op=train_op)
    
  # Add evaluation metrics (for EVAL mode)
  elif mode == tf_estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy':
          tf.metrics.accuracy(
              labels=labels, predictions=tf.argmax(input=logits, axis=1))
    }
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, eval_metric_ops=eval_metric_ops)

def load_mnist():
  (train_data,train_labels), (test_data,test_labels) = tf.keras.datasets.mnist.load_data()

  train_data = np.array(train_data, dtype=np.float32)/255
  test_data = np.array(test_data, dtype=np.float32)/255

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.
  assert train_labels.ndim == 1
  assert test_labels.ndim == 1

  return train_data, train_labels, test_data, test_labels

def main(unused_argv):
  logging.set_verbosity(logging.INFO)

  #load training and test data
  train_data, train_labels, test_data, test_labels = load_mnist()
  
  #instantiate the tf.Estimator
  mnist_classifier = tf_estimator.Estimator(model_fn = cnn_model_fn)

  #Create tf.Estimator input functions for the training and test data.
  train_input_fn = tf_compat_v1_estimator.inputs.numpy_input_fn(
      x={'x':train_data},
      y=train_labels,
      batch_size=FLAGS.batch_size,
      num_epochs=FLAGS.epoch,
      shuffle=True)
  eval_input_fn = tf_compat_v1_estimator.inputs.numpy_input_fn(
      x={'x':test_data}, y=test_labels, num_epochs=1, shuffle=False)
  
  #Training loop.
  steps_per_epoch = 60000//FLAGS.batch_size
  for epoch in range(1, FLAGS.epoch + 1):
    #train the model for one epoch.
    mnist_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

    #Evaluate the model and print results
    eval_results = mnist_classifier.evaluate(input_fn = eval_input_fn)
    test_accuracy = eval_results['accuracy']
    print('Test accuracy after %d epochs is : %.3f'%(epoch, test_accuracy))

if __name__ == '__main__':
  tf.app.run()



I0826 02:20:43.405376 139632686745472 estimator.py:1843] Using default config.
W0826 02:20:43.410086 139632686745472 estimator.py:1865] Using temporary folder as model directory: /tmp/tmp_ipay2np
I0826 02:20:43.414029 139632686745472 estimator.py:202] Using config: {'_model_dir': '/tmp/tmp_ipay2np', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluste

Test accuracy after 1 epochs is : 0.960


I0826 02:21:15.910845 139632686745472 monitored_session.py:243] Graph was finalized.
I0826 02:21:15.913497 139632686745472 saver.py:1395] Restoring parameters from /tmp/tmp_ipay2np/model.ckpt-234
W0826 02:21:15.933686 139632686745472 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/training/saver.py:1161: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
I0826 02:21:15.949723 139632686745472 session_manager.py:527] Running local_init_op.
I0826 02:21:15.956479 139632686745472 session_manager.py:530] Done running local_init_op.
I0826 02:21:16.095124 139632686745472 basic_session_run_hooks.py:629] Calling checkpoint listeners before saving checkpoint 234...
I0826 02:21:16.097084 139632686745472 basic_session_run_hooks.py:633] Saving checkpoints for 234 into /tmp/tmp_ipay2np/model.ckpt.
I0826 02:21:16.14266

Test accuracy after 2 epochs is : 0.971


I0826 02:21:34.430948 139632686745472 monitored_session.py:243] Graph was finalized.
I0826 02:21:34.433861 139632686745472 saver.py:1395] Restoring parameters from /tmp/tmp_ipay2np/model.ckpt-468
I0826 02:21:34.463474 139632686745472 session_manager.py:527] Running local_init_op.
I0826 02:21:34.470915 139632686745472 session_manager.py:530] Done running local_init_op.
I0826 02:21:34.622240 139632686745472 basic_session_run_hooks.py:629] Calling checkpoint listeners before saving checkpoint 468...
I0826 02:21:34.624661 139632686745472 basic_session_run_hooks.py:633] Saving checkpoints for 468 into /tmp/tmp_ipay2np/model.ckpt.
I0826 02:21:34.664574 139632686745472 basic_session_run_hooks.py:641] Calling checkpoint listeners after saving checkpoint 468...
I0826 02:21:34.781604 139632686745472 basic_session_run_hooks.py:266] loss = 0.14109342, step = 468
I0826 02:21:41.854572 139632686745472 basic_session_run_hooks.py:717] global_step/sec: 14.1374
I0826 02:21:41.859650 139632686745472 basi

...

Test accuracy after 9 epochs is : 0.988


I0826 02:23:48.358492 139632686745472 estimator.py:1175] Done calling model_fn.
I0826 02:23:48.364253 139632686745472 basic_session_run_hooks.py:558] Create CheckpointSaverHook.
I0826 02:23:48.455837 139632686745472 monitored_session.py:243] Graph was finalized.
I0826 02:23:48.462986 139632686745472 saver.py:1395] Restoring parameters from /tmp/tmp_ipay2np/model.ckpt-2106
I0826 02:23:48.506117 139632686745472 session_manager.py:527] Running local_init_op.
I0826 02:23:48.516396 139632686745472 session_manager.py:530] Done running local_init_op.
I0826 02:23:48.744770 139632686745472 basic_session_run_hooks.py:629] Calling checkpoint listeners before saving checkpoint 2106...
I0826 02:23:48.749821 139632686745472 basic_session_run_hooks.py:633] Saving checkpoints for 2106 into /tmp/tmp_ipay2np/model.ckpt.
I0826 02:23:48.806167 139632686745472 basic_session_run_hooks.py:641] Calling checkpoint listeners after saving checkpoint 2106...
I0826 02:23:48.977044 139632686745472 basic_session_run

Test accuracy after 10 epochs is : 0.990


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import time

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import estimator as tf_estimator
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib
from tensorflow_privacy.privacy.optimizers import dp_optimizer

"""common python"""
def get_cnn_model(features):
  """Given input features, returns the logits from a simple CNN model."""
  input_layer = tf.reshape(features, [-1, 28, 28, 1])
  y = tf.keras.layers.Conv2D(
      16, 8, strides=2, padding='same', activation='relu')(
          input_layer)
  y = tf.keras.layers.MaxPool2D(2, 1)(y)
  y = tf.keras.layers.Conv2D(
      32, 4, strides=2, padding='valid', activation='relu')(
          y)
  y = tf.keras.layers.MaxPool2D(2, 1)(y)
  y = tf.keras.layers.Flatten()(y)
  y = tf.keras.layers.Dense(32, activation='relu')(y)
  logits = tf.keras.layers.Dense(10)(y)

  return logits


def make_input_fn(split, input_batch_size=256, repetitions=-1, tpu=False):
  """Make input function on given MNIST split."""

  def input_fn(params=None):
    """A simple input function."""
    batch_size = params.get('batch_size', input_batch_size)

    def parser(example):
      image, label = example['image'], example['label']
      image = tf.cast(image, tf.float32)
      image /= 255.0
      label = tf.cast(label, tf.int32)
      return image, label

    dataset = tfds.load(name='mnist', split=split)
    dataset = dataset.map(parser).shuffle(60000).repeat(repetitions).batch(
        batch_size)
    # If this input function is not meant for TPUs, we can stop here.
    # Otherwise, we need to explicitly set its shape. Note that for unknown
    # reasons, returning the latter format causes performance regression
    # on non-TPUs.
    if not tpu:
      return dataset

    # Give inputs statically known shapes; needed for TPUs.
    images, labels = tf.data.make_one_shot_iterator(dataset).get_next()
    # return images, labels
    images.set_shape([batch_size, 28, 28, 1])
    labels.set_shape([
        batch_size,
    ])
    return images, labels

  return input_fn
"""end common.py"""


flags.DEFINE_boolean(
    'dpsgd', True, 'If True, train with DP-SGD. If False, '
    'train with vanilla SGD.')
flags.DEFINE_float('learning_rate', .15, 'Learning rate for training')
flags.DEFINE_float('noise_multiplier', 1.1,
                   'Ratio of the standard deviation to the clipping norm')
flags.DEFINE_float('l2_norm_clip', 1.0, 'Clipping norm')
flags.DEFINE_integer('batch_size', 256, 'Batch size')
flags.DEFINE_integer('epochs', 30, 'Number of epochs')
flags.DEFINE_integer(
    'microbatches', 256, 'Number of microbatches '
    '(must evenly divide batch_size)')
flags.DEFINE_string('model_dir', None, 'Model directory')

FLAGS = flags.FLAGS


def cnn_model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
  """Model function for a CNN."""

  # Define CNN architecture.
  logits = get_cnn_model(features)

  # Calculate loss as a vector (to support microbatches in DP-SGD).
  vector_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits)
  # Define mean of loss across minibatch (for reporting through tf.Estimator).
  scalar_loss = tf.reduce_mean(input_tensor=vector_loss)

  # Configure the training op (for TRAIN mode).
  if mode == tf_estimator.ModeKeys.TRAIN:
    if FLAGS.dpsgd:
      # Use DP version of GradientDescentOptimizer. Other optimizers are
      # available in dp_optimizer. Most optimizers inheriting from
      # tf.compat.v1.train.Optimizer should be wrappable in differentially
      # private counterparts by calling dp_optimizer.optimizer_from_args().
      optimizer = dp_optimizer.DPGradientDescentGaussianOptimizer(
          l2_norm_clip=FLAGS.l2_norm_clip,
          noise_multiplier=FLAGS.noise_multiplier,
          num_microbatches=FLAGS.microbatches,
          learning_rate=FLAGS.learning_rate)
      opt_loss = vector_loss
    else:
      optimizer = tf.compat.v1.train.GradientDescentOptimizer(
          learning_rate=FLAGS.learning_rate)
      opt_loss = scalar_loss

    global_step = tf.compat.v1.train.get_global_step()
    train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)

    # In the following, we pass the mean of the loss (scalar_loss) rather than
    # the vector_loss because tf.estimator requires a scalar loss. This is only
    # used for evaluation and debugging by tf.estimator. The actual loss being
    # minimized is opt_loss defined above and passed to optimizer.minimize().
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode).
  elif mode == tf_estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy':
            tf.compat.v1.metrics.accuracy(
                labels=labels, predictions=tf.argmax(input=logits, axis=1))
    }
    return tf_estimator.EstimatorSpec(
        mode=mode, loss=scalar_loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  logging.set_verbosity(logging.INFO)
  if FLAGS.dpsgd and FLAGS.batch_size % FLAGS.microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

  # Instantiate the tf.Estimator.
  mnist_classifier = tf_estimator.Estimator(
      model_fn=cnn_model_fn, model_dir=FLAGS.model_dir)

  # Training loop.
  steps_per_epoch = 60000 // FLAGS.batch_size
  for epoch in range(1, FLAGS.epochs + 1):
    start_time = time.time()
    # Train the model for one epoch.
    mnist_classifier.train(
        input_fn=make_input_fn('train', FLAGS.batch_size),
        steps=steps_per_epoch)
    end_time = time.time()
    logging.info('Epoch %d time in seconds: %.2f', epoch, end_time - start_time)

    # Evaluate the model and print results
    eval_results = mnist_classifier.evaluate(
        input_fn=make_input_fn('test', FLAGS.batch_size, 1))
    test_accuracy = eval_results['accuracy']
    print('Test accuracy after %d epochs is: %.3f' % (epoch, test_accuracy))

    # Compute the privacy budget expended.
    if FLAGS.dpsgd:
      if FLAGS.noise_multiplier > 0.0:
        eps, _ = compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy(
            60000, FLAGS.batch_size, FLAGS.noise_multiplier, epoch, 1e-5)
        print('For delta=1e-5, the current epsilon is: %.2f' % eps)
      else:
        print('Trained with DP-SGD but with zero noise.')
    else:
      print('Trained with vanilla non-private SGD optimizer')


if __name__ == '__main__':
  app.run(main)

DuplicateFlagError: ignored